## Training Objectives

- **Data Pre-processing**

    - Dummy encoding and One-Hot Encoding in PySpark
    - Train-test Split in PySpark

- **Building a Linear Regression Model in PySpark**
    - Predictive Analysis with ML: Regression (Linear Reression)
    - Model Training and Prediction with PySpark MLlib

- **Model Evaluation**
    
    - Evaluation Metrics: Error
    - Saving, Exporting, and Evaluating Models in MLlib


# 🧠 **Machine Learning**



<img src="assets/supervised_unsupervised.png" width="900">

**Supervised Learning**: 

* memiliki target variable. 
* untuk pembuatan model prediksi (y ~ x)
* ada ground truth (label aktual) sehingga ada evaluasi model

**Unsupervised Learning**: 

* tidak memiliki target variable. 
* untuk mencari pola dalam data sehingga menghasilkan informasi yang berguna/dapat diolah lebih lanjut. umumnya dipakai untuk tahap explanatory data analysis (EDA)/data pre-processing.
* tidak ada ground truth sehingga tidak ada evaluasi model 

# 🛠 **PySpark for Machine Learning**

🌟 **Keunggulan PySpark untuk Machine Learning**

1. **Skalabilitas**:  
   - Mampu memproses data besar dengan komputasi terdistribusi.

2. **Fleksibilitas**:  
   - Mendukung berbagai algoritma seperti klasifikasi, regresi, dan clustering.  

3. **Efisiensi**:  
   - Mempercepat preprocessing, rekayasa fitur, dan pelatihan model.



 ### 🔑 **Komponen Utama**

1️⃣ **Preprocessing Data**
   - Transformasi data mentah menjadi siap pakai, seperti:  
     - Mengubah tipe data
     - Encoding data kategorikal.  

2️⃣ **MLlib (Machine Learning Library)**
   - Mendukung algoritma seperti Linear Regression, Random Forest, Logistic Regression, dan K-Means.  
   - Tersedia alat untuk rekayasa fitur seperti PCA dan StringIndexer.

3️⃣ **Evaluasi Model**
   - Gunakan metrik seperti *error*


<div class="alert alert-info"><br>
  <center><h2>📊 Study Case: Predicting Property Sales Price: in Jakarta, Tangsel, and Depok Area</h2></center>
</div> 

<center>

![house.png](assets/house.png)

</center>

Sebagai Tim Data di sebuah institusi perbankan, diminta untuk melakukan analisis untuk mengetahui prediksi harga properti untuk acuan dasar data credit KPR di perbankan. 

Keinginan untuk memiliki properti sendiri merupakan impian banyak orang. Selain bisa dijadikan tempat tinggal, memiliki properti di Jakarta dan Depok adalah salah satu aset investasi yang menguntungkan karena harganya yang cenderung naik setiap tahunnya.  

Dalam proses pencarian tempat tinggal idaman ini, beberapa orang mungkin saja mengalami hambatan, yaitu kesulitan dalam mencari tempat tinggal yang sesuai dengan spesifikasi yang diinginkan dan budget yang dimiliki. Banyak orang menemukan tempat tinggal dengan harga yang cukup mahal namun tidak sesuai dengan spesifikasi yang ditawarkan.

***“Ada gak sih kira-kira sistem yang dapat memberikan referensi harga properti?”*** menjadi tujuan analisis pada pembahasan analisis kita.


## 🛠 **Load Data**

**Membaca dan Mengeksplorasi Data**:
   - Menggunakan dataset dari `data_input/property.csv` untuk analisis.
   - Data ini berisi informasi penting tentang harga properti di Jakarta, Tangerang dan Depok.


In [55]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Machine Learning with PySpark").getOrCreate()

In [56]:
# Read data
properti = spark.read.csv('data_input/property.csv', header=True, inferSchema=True)

In [57]:
properti.show(5)

+-----------+-----------+-------------+--------------------+-------------+---------------+----------+
|kamar_mandi|kamar_tidur|luas_bangunan|          sertifikat|tipe_properti|           kota|     harga|
+-----------+-----------+-------------+--------------------+-------------+---------------+----------+
|          3|          4|          294|SHM - Sertifikat ...|        Rumah|  Jakarta Utara|3500000000|
|          3|          3|           78|SHM - Sertifikat ...|    Apartemen|Jakarta Selatan|2500000000|
|          1|          1|           33|HGB - Hak Guna Ba...|    Apartemen|  Jakarta Timur| 265000000|
|          2|          2|          120|SHM - Sertifikat ...|        Rumah|  Jakarta Pusat|2600000000|
|          3|          3|          130|SHM - Sertifikat ...|        Rumah|          Depok|1300000000|
+-----------+-----------+-------------+--------------------+-------------+---------------+----------+
only showing top 5 rows



Berikut deskripsi variabel dari data tersebut:
- `kamar_mandi`: Jumlah kamar mandi pada suatu properti

- `kamar_tidur`: Jumlah kamar tidur pada suatu properti

- `luas_bangunan`: Luas bangunan properti (m2)

- `sertifikat`: Jenis sertifikat atas properti yang di jual
    - `Sertifikat Hak Pakai`
    - `Sertifikat Hak Sewa`
    - `Sertifikat lainnya (PPJB, Girik, Adat, dll)`
    - `Sertifikat PPJB`
    - `Sertifikat Hak Milik`
    
- `tipe_properti`: Jenis properti yang dijual
    - `Rumah`: Untuk tipe properti rumah
    - `Apartemen`: Untuk tipe properti apartemen
    
- `kota`: Lokasi kota tempat properti di jual
    - `Depok`
    - `Jakarta Selatan`
    - `Jakarta Timur`
    - `Jakarta Utara`
    - `Jakarta Barat`
    - `Jakarta Pusat`
    - `Tangerang Selatan`
- `harga`: Nominal harga properti yang dijual

🔍 **Tujuan**

menggunakan data ini untuk:
- **Memprediksi** harga properti di wilayah Jakarta, Depok dan Tangerang berdasarkan beberapa variable prediktor.

Namun, sebelum melakukan analisis, kita harus **membersihkan dan memproses data** agar memiliki format dan kualitas yang sesuai untuk model klasifikasi risiko kredit.


## 📊 **Data Preprocessing**

### Dummy Encoding dan One-Hot Encoding di PySpark

**Kebutuhan Model**: Model machine learning hanya dapat bekerja dengan data dalam format numerik. Fitur kategorikal harus diubah menjadi representasi numerik agar model bisa memahami dan menganalisis data.

Oleh karena itu, perlu mengubah variabel kategorikal ini menjadi format numerik. Di PySpark, dapat menggunakan `StringIndexer` dan `OneHotEncoder` untuk mengonversi data kategorikal menjadi format dummy atau one-hot encoded.

🛠 **Sintaks Dasar untuk Encoding**

1. **Impor Library**:
   ```python
   from pyspark.ml.feature import StringIndexer, OneHotEncoder
   ```

2. **Inisialisasi StringIndexer**:
   - Mengubah teks menjadi indeks numerik.
   ```python
   indexer = StringIndexer(inputCol="col", outputCol="col_index")
   ```

3. **Inisialisasi OneHotEncoder**:
   - Mengubah indeks menjadi representasi one-hot.
   ```python
   encoder = OneHotEncoder(inputCol="col_index", outputCol="col_encoded")
   ```

🔄 **Proses Encoding untuk Beberapa Kolom Kategorikal**

In [58]:
properti.show(3)

+-----------+-----------+-------------+--------------------+-------------+---------------+----------+
|kamar_mandi|kamar_tidur|luas_bangunan|          sertifikat|tipe_properti|           kota|     harga|
+-----------+-----------+-------------+--------------------+-------------+---------------+----------+
|          3|          4|          294|SHM - Sertifikat ...|        Rumah|  Jakarta Utara|3500000000|
|          3|          3|           78|SHM - Sertifikat ...|    Apartemen|Jakarta Selatan|2500000000|
|          1|          1|           33|HGB - Hak Guna Ba...|    Apartemen|  Jakarta Timur| 265000000|
+-----------+-----------+-------------+--------------------+-------------+---------------+----------+
only showing top 3 rows



In [59]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder

1️⃣ **List Kolom Kategorikal**: Menentukan kolom mana yang akan diproses.

In [60]:
# List of categorical columns
cat_columns = ['sertifikat','tipe_properti','kota']

2️⃣ **Buat List Kosong untuk Menyimpan Tahapan Pipeline**: 
   - memerlukan dua list, satu untuk menyimpan tahap pengindeksan dan satu lagi untuk pengkodean.

In [61]:
# Create empty lists to store the stages of the pipeline
indexers = []
encoders = []

3️⃣ **Iterasi Melalui Setiap Kolom Kategorikal**:
   - Menggunakan loop untuk membuat dan menyimpan `StringIndexer` dan `OneHotEncoder` untuk setiap kolom.
   - Dengan mengiterasi kolom, bisa otomatis memproses semua kolom kategorikal tanpa harus menuliskan kode berulang kali untuk setiap kolom.

In [62]:
# Iterate over each categorical column for indexing and encoding
for col in cat_columns:
    # Create a StringIndexer for each column
    indexer = StringIndexer(inputCol=col, outputCol=f"{col}_index", stringOrderType='alphabetAsc', handleInvalid="keep")
    indexers.append(indexer)
    
    # Create an OneHotEncoder for the indexed column
    encoder = OneHotEncoder(inputCol=f"{col}_index", outputCol=f"{col}_encoded")
    encoders.append(encoder)

In [63]:
encoders

[OneHotEncoder_e0ad33712789,
 OneHotEncoder_701e63df918a,
 OneHotEncoder_c14e963eed18]

4️⃣ **Membuat Pipeline**

- **Simplifikasi Proses**: Pipeline memungkinkan untuk melakukan beberapa transformasi secara berurutan dalam satu langkah, sehingga proses menjadi lebih teratur dan mudah dikelola.
- **Gabungkan Indexers dan Encoders**: Mengatur tahapan encoding dalam satu workflow.

In [64]:
from pyspark.ml import Pipeline

# Combine indexers and encoders into a single pipeline
pipeline = Pipeline(stages = indexers + encoders)

5️⃣ **Fit Pipeline untuk Membuat Fitur yang Dikonversi**: Menerapkan seluruh tahapan dalam satu langkah.

In [65]:
properti.show(3)

+-----------+-----------+-------------+--------------------+-------------+---------------+----------+
|kamar_mandi|kamar_tidur|luas_bangunan|          sertifikat|tipe_properti|           kota|     harga|
+-----------+-----------+-------------+--------------------+-------------+---------------+----------+
|          3|          4|          294|SHM - Sertifikat ...|        Rumah|  Jakarta Utara|3500000000|
|          3|          3|           78|SHM - Sertifikat ...|    Apartemen|Jakarta Selatan|2500000000|
|          1|          1|           33|HGB - Hak Guna Ba...|    Apartemen|  Jakarta Timur| 265000000|
+-----------+-----------+-------------+--------------------+-------------+---------------+----------+
only showing top 3 rows



In [66]:
# Fit the pipeline to create the encoded features
pipeline_model = pipeline.fit(properti)

In [67]:
properti = pipeline_model.transform(properti)

In [68]:
properti.show(5)

+-----------+-----------+-------------+--------------------+-------------+---------------+----------+----------------+-------------------+----------+------------------+---------------------+-------------+
|kamar_mandi|kamar_tidur|luas_bangunan|          sertifikat|tipe_properti|           kota|     harga|sertifikat_index|tipe_properti_index|kota_index|sertifikat_encoded|tipe_properti_encoded| kota_encoded|
+-----------+-----------+-------------+--------------------+-------------+---------------+----------+----------------+-------------------+----------+------------------+---------------------+-------------+
|          3|          4|          294|SHM - Sertifikat ...|        Rumah|  Jakarta Utara|3500000000|             5.0|                1.0|       5.0|     (6,[5],[1.0])|        (2,[1],[1.0])|(7,[5],[1.0])|
|          3|          3|           78|SHM - Sertifikat ...|    Apartemen|Jakarta Selatan|2500000000|             5.0|                0.0|       3.0|     (6,[5],[1.0])|        (2,[

### 📦 Splitting Data into Training and Test Sets

Untuk mengembangkan model yang dapat diandalkan, perlu membagi data menjadi set pelatihan dan pengujian. Pembagian ini adalah langkah sederhana namun sangat penting dalam machine learning.

🤔 **Mengapa Kita Membagi Data?**

- **Generalitas Model**: Proses ini memastikan bahwa model dapat menggeneralisasi dengan baik ke data yang belum pernah dilihat. 
- **Pembelajaran dan Evaluasi**: 
  - **Set Pelatihan (Train)**: Digunakan untuk melatih model, membantunya mempelajari pola dari data.
  - **Set Pengujian (Test)**: Digunakan untuk mengevaluasi seberapa baik model dapat menerapkan pola tersebut pada data baru, sehingga membantu mencegah overfitting.

![split.png](assets/test-trainn.png)

🔀 Splitting Data
Kita bisa membagi data menggunakan metode `randomSplit` di PySpark.

Tentukan beberapa parameter:

- **Proposisi Split**: 
  - `[0.8, 0.2]` menunjukkan bahwa 80% dari data akan digunakan untuk pelatihan dan 20% untuk pengujian.
- **Seed**
  - `seed=123` memastikan bahwa pembagian data bersifat reproduktif. Ini artinya jika kita menjalankan kode yang sama di lain waktu, kita akan mendapatkan pembagian yang sama.

In [69]:
# splitting data
train_data, test_data = properti.randomSplit([0.8, 0.2], seed = 123)

In [70]:
train_data.count()

5015

In [71]:
test_data.count()

1272

In [72]:
train_data.show(3)

+-----------+-----------+-------------+--------------------+-------------+-------------+---------+----------------+-------------------+----------+------------------+---------------------+-------------+
|kamar_mandi|kamar_tidur|luas_bangunan|          sertifikat|tipe_properti|         kota|    harga|sertifikat_index|tipe_properti_index|kota_index|sertifikat_encoded|tipe_properti_encoded| kota_encoded|
+-----------+-----------+-------------+--------------------+-------------+-------------+---------+----------------+-------------------+----------+------------------+---------------------+-------------+
|          1|          1|           14|HGB - Hak Guna Ba...|    Apartemen|Jakarta Utara|290000000|             0.0|                0.0|       5.0|     (6,[0],[1.0])|        (2,[0],[1.0])|(7,[5],[1.0])|
|          1|          1|           14|HGB - Hak Guna Ba...|    Apartemen|Jakarta Utara|290000000|             0.0|                0.0|       5.0|     (6,[0],[1.0])|        (2,[0],[1.0])|(7,[5


### 📈 Feature Selection

Setelah melakukan beberapa tahapan pre-procesing data, langkah selanjutnya sebelum masuk dalam pemodelan adalah memilih beberapa fitur (kolom) yang akan dijadikan sebagai prediktor untuk melakukan prediksi harga properti.

**1️⃣ List Kolom Prediktor**  

Identifikasi kolom prediktor mana yang ingin kita gunakan.

In [73]:
feature_predictor = ['kamar_mandi','kamar_tidur','luas_bangunan','sertifikat_encoded','tipe_properti_encoded','kota_encoded']

**2️⃣ Menggabungkan Semua Fitur** 

Fitur-fitur yang sudah dipilih akan digabungkan menjadi satu kesatuan vector dengan menggunakan fungsi `VectorAssembler()`.

**🤔 Mengapa Perlu `VectorAssembler()`?**

`VectorAssembler()` diperlukan dikarenakan pemrosesan machine learning pada PySpark hanya menerima input data dalam bentuk vector.

🛠 **Sintaks untuk mengubah dalam bentuk vector**

   ```python
   VectorAssembler(inputCols = kolom_predictor, outputCol = 'nama_kolom_hasil')
   ```

In [74]:
from pyspark.ml.feature import VectorAssembler

In [75]:
assembler = VectorAssembler(inputCols = feature_predictor,
                            outputCol = 'predictors')

In [76]:
assembler

VectorAssembler_d565c306e122

In [87]:
#train_data.show(truncate = False)

**3️⃣ Transformasi Data**  
Terapkan assembler pada kedua data train dan test. Penting untuk memastikan keduanya diubah dengan cara yang sama.


In [88]:
# Transformasi data training dan test menggunakan assembler
train_data = assembler.transform(train_data)
test_data = assembler.transform(test_data)

In [93]:
#train_data.select('predictors').show(5, truncate=False)


## 📈 Training Model Regression

### Konsep Linear Regression

Regresi Linear (Linear Regression) merupakan salah satu algoritma sederhana pada machine learning yang digunakan untuk memprediksi nilai numerik seperti harga rumah, profit, credit score dan lain sebagianya.

Dalam linear regresi, terdapat istilah variable `y` (target) dan variable `x` (predictor). Cara kerja algoritma ini adalah dengan cara melakukan prediksi nilai `y` berdasarkan variable `x`. Dimana kita dapat juga menyebut variable `y` sebagai variable `dependent` dan variable `y` adalah variable `independent`.

Linear Regression termasuk dalam golongan interpretable model, yang dimana kita dapat melakukan interpretasi hasil prediksi berdasarkan komponen variable `x` terhadap `y`

![](assets/lr.png)

🧪 **Formula model linear regression:**

$$
\hat{y}=\beta_0+\beta_1.x_1+\beta_2.x_2+\beta_3.x_3+\beta_4.x_4+ ....
$$

dimana:
- $\hat{y}$ : nilai prediksi target variabel
- $\beta_0$ : nilai intercept (nilai target variabel ketika kita tidak memiliki prediktor sama sekali)
- $\beta_1, \beta_2, dst$ : nilai slope (nilai kemiringan garis regresi / nilai kontribusi prediktor dalam menentukan target variabel)



### 🏗️ Building and Training a Linear Regression Model

#### 1️⃣ 🔧 Membangun Model Linear Regression

menggunakan `LinearRegression` untuk membangun model Random Forest kita. Dalam model ini, kita akan menetapkan beberapa parameter:

1.  `labelCol`: Diisi dengan kolom target, yaitu kolom `harga`
2.  `featuresCol`: Diisi dengan kolom kategorical yang telah di encoding dan juga kolom numerikal. Yaitu kolom predictor hasil assembler
3.  `predictionCol`: Diisi dengan nama kolom untuk menampung hasil prediksi

In [91]:
train_data.show(3)

+-----------+-----------+-------------+--------------------+-------------+-------------+---------+----------------+-------------------+----------+------------------+---------------------+-------------+--------------------+
|kamar_mandi|kamar_tidur|luas_bangunan|          sertifikat|tipe_properti|         kota|    harga|sertifikat_index|tipe_properti_index|kota_index|sertifikat_encoded|tipe_properti_encoded| kota_encoded|          predictors|
+-----------+-----------+-------------+--------------------+-------------+-------------+---------+----------------+-------------------+----------+------------------+---------------------+-------------+--------------------+
|          1|          1|           14|HGB - Hak Guna Ba...|    Apartemen|Jakarta Utara|290000000|             0.0|                0.0|       5.0|     (6,[0],[1.0])|        (2,[0],[1.0])|(7,[5],[1.0])|(18,[0,1,2,3,9,16...|
|          1|          1|           14|HGB - Hak Guna Ba...|    Apartemen|Jakarta Utara|290000000|          

In [92]:
from pyspark.ml.regression import LinearRegression

# Define the LinearRegression
lr = LinearRegression(labelCol = 'harga', #target variable data actual
                      featuresCol = 'predictors', #predictor hasil vectorAssembler
                      predictionCol = 'harga_predict') #kolom baru yang menyimpan hasil prediksi

#### 2️⃣ 🏋️‍♂️ Melatih Model

Setelah mendefinisikan model linear regression, dapat melatih model pada data train

In [94]:
model = lr.fit(train_data)

#### 3️⃣ 🔍Menampilkan Hasil Summary dan Interpretasi Hasil

In [98]:
# code here
print("Coefficients:", model.coefficients)

Coefficients: [89408993.78095956,96851936.27514121,13892010.2042854,22154944.841700613,-157453793.99205738,-271412855.6324927,51815433.256719254,-146432660.74914736,-45701543.639278725,168749312.48917338,-168749312.4891851,-304363960.96230525,118852325.953356,210172966.64795232,509215726.21113896,-305578593.5357101,52841398.76475487,-42027006.45047871]


In [99]:
# code here
model.intercept

20884134.357347567

### [Additional] Menampilkan Keseluruhan Coefficient

In [100]:
# Mendapatkan nama prediktor
encoded_cat_cols = ["sertifikat_HGB - Hak Guna Bangunan","sertifikat_HP - Hak Pakai", "sertifikat_HS - Hak Sewa", "sertifikat_Lainnya (PPJB,Girik,Adat,dll)",\
                    "sertifikat_PPJB", "sertifikat_SHM - Sertifikat Hak Milik", "tipe_properti_Apartemen", "tipe_properti_Rumah", \
                    "kota_Depok", "kota_Jakarta Barat", "kota_Jakarta Pusat", "kota_Jakarta Selatan","kota_Jakarta Timur","kota_Jakarta Utara","kota_Tangerang Selatan"] 
numeric_cols = ["kamar_mandi", "kamar_tidur","luas_bangunan"]  
input_cols =  numeric_cols + encoded_cat_cols

# Mendapatkan koefisien dan menghubungkannya dengan nama prediktor
coefficients = model.coefficients
intercept = model.intercept

coef_table = [(input_cols[i], coefficients[i]) for i in range(len(coefficients))]
coef_table.append(("Intercept", intercept))

# Menampilkan tabel
print("Koefisien Model:")
print("{:<15} {:<10}".format("Predictor", "Coefficient"))
print("-" * 25)
for predictor, coef in coef_table:
    print(f"{predictor:<15} {coef:<10.4f}")

Koefisien Model:
Predictor       Coefficient
-------------------------
kamar_mandi     89408993.7810
kamar_tidur     96851936.2751
luas_bangunan   13892010.2043
sertifikat_HGB - Hak Guna Bangunan 22154944.8417
sertifikat_HP - Hak Pakai -157453793.9921
sertifikat_HS - Hak Sewa -271412855.6325
sertifikat_Lainnya (PPJB,Girik,Adat,dll) 51815433.2567
sertifikat_PPJB -146432660.7491
sertifikat_SHM - Sertifikat Hak Milik -45701543.6393
tipe_properti_Apartemen 168749312.4892
tipe_properti_Rumah -168749312.4892
kota_Depok      -304363960.9623
kota_Jakarta Barat 118852325.9534
kota_Jakarta Pusat 210172966.6480
kota_Jakarta Selatan 509215726.2111
kota_Jakarta Timur -305578593.5357
kota_Jakarta Utara 52841398.7648
kota_Tangerang Selatan -42027006.4505
Intercept       20884134.3573


**💡 Interpretasi:**

- Setiap kenaikan 1m2 luas bangunan akan meningkatkan harga rumah sebesar 13892010.2043
- Property yang dibangun di jakarta utara memiliki potensi kenaikan harga sekitar 52841398.7648

## 🔮 Melakukan Prediksi

Tujuan utama dalam membangun model machine learning adalah untuk memprediksi hasil dengan error yang rendah berdasarkan data baru. Dalam hal ini, kita akan menggunakan model Linear Regresi yang telah dilatih untuk melakukan prediksi pada dataset test, data yang belum pernah dilihat sebelumnya untuk mengevaluasi kinerja dan efektivitasnya.

### Langkah-Langkah Melakukan Prediksi di PySpark

1️⃣ **Melakukan Prediksi**:
   Kita menggunakan fungsi `transform` untuk melakukan prediksi pada data test.

In [101]:
# Make predictions on the test data
predictions = model.transform(test_data)

2️⃣ **Menampilkan Hasil Prediksi**: Kita dapat menampilkan hasil prediksi bersama dengan data aktualnya.

In [103]:
test_data.show(3)

+-----------+-----------+-------------+--------------------+-------------+-------------+---------+----------------+-------------------+----------+------------------+---------------------+-------------+--------------------+
|kamar_mandi|kamar_tidur|luas_bangunan|          sertifikat|tipe_properti|         kota|    harga|sertifikat_index|tipe_properti_index|kota_index|sertifikat_encoded|tipe_properti_encoded| kota_encoded|          predictors|
+-----------+-----------+-------------+--------------------+-------------+-------------+---------+----------------+-------------------+----------+------------------+---------------------+-------------+--------------------+
|          1|          1|           14|HGB - Hak Guna Ba...|    Apartemen|Jakarta Utara|290000000|             0.0|                0.0|       5.0|     (6,[0],[1.0])|        (2,[0],[1.0])|(7,[5],[1.0])|(18,[0,1,2,3,9,16...|
|          1|          1|           14|Lainnya (PPJB,Gir...|    Apartemen|Jakarta Utara|290000000|          

In [102]:
# code here
predictions.show(4)

+-----------+-----------+-------------+--------------------+-------------+-------------+---------+----------------+-------------------+----------+------------------+---------------------+-------------+--------------------+-------------------+
|kamar_mandi|kamar_tidur|luas_bangunan|          sertifikat|tipe_properti|         kota|    harga|sertifikat_index|tipe_properti_index|kota_index|sertifikat_encoded|tipe_properti_encoded| kota_encoded|          predictors|      harga_predict|
+-----------+-----------+-------------+--------------------+-------------+-------------+---------+----------------+-------------------+----------+------------------+---------------------+-------------+--------------------+-------------------+
|          1|          1|           14|HGB - Hak Guna Ba...|    Apartemen|Jakarta Utara|290000000|             0.0|                0.0|       5.0|     (6,[0],[1.0])|        (2,[0],[1.0])|(7,[5],[1.0])|(18,[0,1,2,3,9,16...|6.453788633690729E8|
|          1|          1|   

In [104]:
predictions.select('harga','harga_predict').show(4)

+---------+-------------------+
|    harga|      harga_predict|
+---------+-------------------+
|290000000|6.453788633690729E8|
|290000000|6.750393517840915E8|
|290000000|6.750393517840915E8|
|300000000|6.750393517840915E8|
+---------+-------------------+
only showing top 4 rows



## 📊 Evaluasi Model

Tujuan memahami seberapa baik model kita dapat memprediksi data. Setelah melatih model dan membuat prediksi, sangat penting untuk mengevaluasi kinerjanya guna menentukan efektivitasnya. Salah satu metrik kunci untuk mengevaluasi model regresi adalah **error**.

### Error

Untuk melihat apakah prediksi yang dibuat menghasilkan nilai error terkecil
  
**Error/residual adalah selisih antara hasil prediksi dengan nilai aktual.**

$$
Error/residual = actual - prediction = y - \hat y
$$

Terdapat beberapa nilai error yang ada :

1. MAE (Mean Absolute Error): Memperlakukan error dengan lebih ringan. **Formula:**
   $$
   MAE = \frac{1}{N} \sum_{i=1}^{N} \left | y_{i} - \hat{y} \right |
   $$

1. RMSE (Root Mean Square Error): Memperlakukan error dengan lebih sensitif. Ketika nilai error besar, maka nilai RMSE akan semakin besar dan sebaliknya. **Formula:**
   $$
   RMSE = \sqrt{\frac{1}{N} \sum_{i=1}^{N} (y_{i} - \hat{y})^{2}}
   $$
     
   
RMSE digunakan ketika model yang dibuat memuat observasi outlier. Sedangkan, MAE digunakan ketika model yang dibuat tidak memuat observasi outlier

Untuk menampilkan nilai MAE dan RMSE kita dapat memanfaatkan variable `summary_hasil` yang berisi rangkuman hasil modelling.
  

In [105]:
summary_hasil = model.summary

In [106]:
# MAE
summary_hasil.meanAbsoluteError

483415812.62103844

In [107]:
# RMSE
summary_hasil.rootMeanSquaredError

704687635.0882933

In [109]:
# R-Squared
summary_hasil.r2

0.680327488241768

In [108]:
# Adjusted R-Squared
summary_hasil.r2adj

0.6791757458054893

**💡 Interpretasi:**

- 
- 

#### **Additional:MAPE**

3. MAPE (Mean Absolute Percentage Error): Menunjukan seberapa besar penyimpangan error dalam bentuk persentase
   $$
   MAPE = \frac{1}{N} \sum_{i=1}^{N} \frac {\left | y_{i} - \hat{y} \right |} {y}
   $$

In [ ]:
# from pyspark.sql.functions import abs, col, mean

# mape_df = predictions.withColumn(
#     "absolute_percentage_error",
#     abs((col("harga") - col("harga_predict")) / col("harga")) * 100
# )
# mape = mape_df.select(mean("absolute_percentage_error")).collect()[0][0]

# print(f"MAPE: {mape:.2f}%")